# Загрузка текстов

In [ ]:
import requests

In [ ]:
resp = requests.get('https://api.hh.ru/vacancies/1')

In [ ]:
# вакансии IT
url1 = 'https://api.hh.ru/vacancies?specialization=1.221'

In [ ]:
# вакансии фармацевтика
url2 = 'https://api.hh.ru/vacancies?specialization=13.228'

In [ ]:
searches = []
for page in range(20):
    searches.append(
        requests.get(url1 + '&page={}&per_page=100'.format(page)).json()
    )

In [ ]:
len(searches)

In [ ]:
for page in range(20):
    searches.append(
        requests.get(url2 + '&page={}&per_page=100'.format(page)).json()
    )

In [ ]:
searches[0]['items'][0]['url']

In [ ]:
vacs = [x['items'] for x in searches]

In [ ]:
len(vacs[0])

In [ ]:
vacs = sum(vacs, [])

In [ ]:
urls = [x['url'] for x in vacs]

In [ ]:
vac_dicts = []
for url in urls:
    vac_dicts.append(requests.get(url).json())

In [ ]:
import pickle
with open('vac_dicts.pkl', 'wb') as f:
    pickle.dump(vac_dicts, f)

Скачивание происходит достаточно долго, поэтому возьмем здесь архив: https://www.dropbox.com/s/9do3k7g24px857j/vac_dicts.zip?dl=0 и распакуем его.

In [ ]:
import pickle
with open('vac_dicts.pkl', 'rb') as f:
    data = pickle.load(f)

In [ ]:
from bs4 import BeautifulSoup
texts = []
for v in data:
    texts.append(
        BeautifulSoup(v['description']).text
    )

# Похожесть текстов. Косинусная близость

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# http://pymorphy2.readthedocs.io/en/latest/_modules/pymorphy2/tokenizers.html

import re
GROUPING_SPACE_REGEX = re.compile(r'([^\w]|[+])', re.UNICODE)
def simple_word_tokenize(text, _split=GROUPING_SPACE_REGEX.split):
    return [t for t in _split(text.lower()) if t and not t.isspace()]

In [ ]:
import pymorphy2
m = pymorphy2.MorphAnalyzer()

In [ ]:
def token_r(text):
    words = simple_word_tokenize(text)
    return [m.parse(x)[0].normal_form for x in words if len(x) >= 4]

In [ ]:
cv = CountVectorizer(tokenizer=token_r)

In [ ]:
%%time
matrix = cv.fit_transform(texts).toarray()

In [ ]:
matrix.shape

In [ ]:
matrix[0]

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
cos_m = cosine_similarity(matrix)

In [ ]:
cos_m.shape

In [ ]:
%matplotlib inline

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.plot(cos_m[0][1:])

Различие уже видно невооруженным взглядом, но оно не так ярко выражено. Идеи по улучшению?